In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
import os
import pandas as pd
import numpy as np

In [4]:
TRAIN_DIR='images/train'
TEST_DIR='images/validation'


In [5]:
def createDataFrame(dir):
    images_path=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            images_path.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return images_path,labels
        

In [6]:
train=pd.DataFrame()
train['image'],train['label']=createDataFrame(TRAIN_DIR)




angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
test=pd.DataFrame()
test['image'],test['label']=createDataFrame(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(train
     )

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [9]:
train.shape

(28821, 2)

In [10]:
from tqdm.notebook import tqdm
def extract_features(images):
    features=[]
    for image in tqdm(images):
        img=load_img(image,color_mode='grayscale')
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features
        
    

In [11]:
train_features=extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [13]:
test_features=extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [14]:
X_train=train_features/255.0
X_test=test_features/255.0

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
le=LabelEncoder()

In [19]:
le.fit(train['label'])

LabelEncoder()

In [20]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [21]:
y_train

array([0, 0, 0, ..., 6, 6, 6])

In [22]:
y_train.shape

(28821,)

In [25]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

(28821, 7)

In [23]:
model=Sequential()

In [32]:
#convolutional layers

model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))


#flatten

model.add(Flatten())

model.add(Dense(512,activation='relu',))
model.add(Dropout(0.4))

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(7,activation='softmax'))

In [36]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [37]:
history=model.fit(x=X_train,y=y_train,epochs=100,batch_size=512,validation_data=(X_test,y_test))

Epoch 1/100
57/57 [==============================] - 418s 7s/step - loss: 1.8280 - accuracy: 0.2406 - val_loss: 1.8095 - val_accuracy: 0.2583
Epoch 2/100
57/57 [==============================] - 411s 7s/step - loss: 1.8144 - accuracy: 0.2473 - val_loss: 1.8004 - val_accuracy: 0.2583
Epoch 3/100
57/57 [==============================] - 400s 7s/step - loss: 1.7960 - accuracy: 0.2491 - val_loss: 1.7829 - val_accuracy: 0.2594
Epoch 4/100
57/57 [==============================] - 414s 7s/step - loss: 1.7505 - accuracy: 0.2714 - val_loss: 1.7144 - val_accuracy: 0.3109
Epoch 5/100
57/57 [==============================] - 415s 7s/step - loss: 1.6520 - accuracy: 0.3350 - val_loss: 1.5290 - val_accuracy: 0.3951
Epoch 6/100
57/57 [==============================] - 415s 7s/step - loss: 1.5129 - accuracy: 0.4081 - val_loss: 1.4090 - val_accuracy: 0.4585
Epoch 7/100
57/57 [==============================] - 394s 7s/step - loss: 1.4265 - accuracy: 0.4467 - val_loss: 1.3150 - val_accuracy: 0.4977
Epoch 

KeyboardInterrupt: 

In [53]:
def ef(image):
    img=load_img(image,color_mode='grayscale')
    features=np.array(img)
    features=features.reshape(1,48,48,1)
    return features/255.0

In [54]:
label=['angry','disgust','fear','happy','neutral','sad','suprise']

In [56]:
image='images/validation/sad/10116.jpg'
print("orginal image sad")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("model prediction : ",pred_label)


orginal image sad
1/1 [==============================] - 0s 26ms/step
model prediction :  sad


In [45]:
model.predict

<bound method Model.predict of <keras.src.engine.sequential.Sequential object at 0x000002502FA3BF50>>

In [61]:
model_json =model.to_json()
with open('emotionDetect.json','w') as json_file:
    json_file.write(model_json)
model.save('emotiondetect.h5')

C:\Users\Shalender\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [63]:
from keras.models import model_from_json

In [ ]:
# NOTE:
#     emotionDetect has accuracy of 63 percentage
    
#     whereas :
#         emotionDetector has an accuracy of 74 percentage 